In [2]:
!pip install tensorflow-gpu

from google.colab import drive
drive.mount('/content/gdrive')
!cp gdrive/My\ Drive/*.npy .
!cp gdrive/My\ Drive/*.py .
!ls -l

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
total 5395456
-rw------- 1 root root       2875 Mar 21 14:18 _dataset_tools.py
drwx------ 3 root root       4096 Mar 21 14:12 gdrive
-rw------- 1 root root       2398 Mar 21 14:18 _my_tools.py
drwxr-xr-x 1 root root       4096 Mar  8 17:26 sample_data
-rw------- 1 root root  920825984 Mar 21 14:12 X_test.npy
-rw------- 1 root root 2762440832 Mar 21 14:16 X_train.npy
-rw------- 1 root root  460413056 Mar 21 14:17 y_test.npy
-rw------- 1 root root 1381220480 Ma

In [1]:
import tensorflow as tf
from tensorflow import keras
import _my_tools as mt
import numpy as np

# X_train, y_train, X_test, y_test = mt.loadData("",'float16',channels_last=True)

print("tf version - ", tf.__version__)

tf version -  1.13.1


In [33]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer, InputSpec

# It assumes, that we are using channel_last.
class SepConv(Layer):

    def __init__(self, **kwargs):
        super(SepConv, self).__init__(**kwargs)
        self.input_spec = [InputSpec(ndim=4), InputSpec(ndim=4),InputSpec(ndim=4)]

        self.sess = tf.Session()
#         self.input_spec = [InputSpec(ndim=4)]

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        assert (len(input_shape) == 3)
        print(input_shape)
        self.input_spec = [InputSpec(shape=input_shape[0]), InputSpec(shape=input_shape[1]), InputSpec(shape=input_shape[2])]
#         print(self.input_spec[0].shape)
        self.checkInput()
#         super(SepConv, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        assert isinstance(x, list)
        assert (len(x) == 3)
        
#         print(img)
#         c = K.shape(x[0])
#         print("A",self.m_out)
#         output = np.zeros([1, self.m_out, self.m_out, self.n_channels])
#         print("out - ",output.shape)
#         print(output[0,0,0,:])
#         print("B")
#         output = tf.map_fn(lSC,[])
#         for b in range(self.n_b): # now it's not paralelized. It will be probably slow.
#             self.localSepConv(img[b],horizontal[b],vertical[b],output=output[b])
        
        output = tf.map_fn(self.l_1, x, dtype=tf.float32)
        
        return output

    def compute_output_shape(self, input_shape):
        assert isinstance(input_shape, list)
        assert (len(input_shape) == 3)

        img, horizontal, vertical = input_shape
#         self.checkInput()
        
        return [self.n_b, self.m_out, self.m_out, self.n_channels]

    def checkInput(self):#, img, horizontal, vertical):
#         i_shape = keras.backend.shape(img)
#         h_shape = keras.backend.shape(horizontal)
#         v_shape = keras.backend.shape(vertical)
        
        i_shape = self.input_spec[0].shape
        h_shape = self.input_spec[1].shape
        v_shape = self.input_spec[2].shape
    
        self.n_b = i_shape[0] # number of batches
        self.m = i_shape[1] # dimension of image
        self.n_channels = i_shape[3] # number of channels
        self.filter_size = h_shape[3] # size of filter == depth of NN output
        self.m_out = self.m - self.filter_size + 1
        
        print("here - ",i_shape[1])
        
#         tf.assert_equal(i_shape[1],i_shape[2])
        assert i_shape[1] == i_shape[2]
        assert h_shape[3] == v_shape[3]
        assert h_shape[1] == h_shape[2] == v_shape[1] == v_shape[2] == self.m_out
    
    def localSepConv(self, x):
        img, horizontal, vertical = x
        output = np.zeros([self.m_out, self.m_out, self.n_channels])
        
        for row in range(self.m_out):
            for col in range(self.m_out):
                sub_patch = img[row:row + self.filter_size, col:col + self.filter_size, :]
#                 print(sess.run(tf.shape(sub_patch)))
                kernel_h = tf.reshape( horizontal[row,col,:] , [1,self.filter_size,self.n_channels] )
#                 print(sess.run(kernel_h))
#                 kernel_v = tf.transpose(vertical[row,col,:])
                kernel_v = tf.reshape( vertical[row,col,:] , [self.filter_size,1,self.n_channels] )
#                 print(sess.run(kernel_v))
#                 print(sess.run( tf.shape(tf.reduce_sum(sub_patch * kernel_h * kernel_v,[0,1]) )))
                output[row,col,:] = tf.reduce_sum(sub_patch * kernel_h * kernel_v,[0,1])#.eval(session=self.sess)
                
        return tf.convert_to_tensor(output)
    
    def l_1(self, x):
        img, horizontal, vertical = x
        
        r = tf.convert_to_tensor(list(range(self.m_out)))
        
        output = tf.map_fn(lambda x: self.l_2(img,x), [horizontal,vertical,r], dtype=tf.float32)
        
        return output
        
    def l_2(self, img, x):
        horizontal, vertical, i = x

        r = tf.convert_to_tensor(list(range(self.m_out)))
        
        output = tf.map_fn(lambda x: self.l_3(img[i:i + self.filter_size,:,:],x),[horizontal, vertical,r], dtype=tf.float32)
        
        return output
    
    def l_3(self, img, x):
        horizontal, vertical, i = x
        
        sub_patch = img[:,i:i + self.filter_size,:]
        kernel_h = tf.reshape(horizontal,[1, self.filter_size, 1])
        kernel_v = tf.reshape(vertical, [self.filter_size, 1, 1])
        
        output = tf.reduce_sum(sub_patch * kernel_h * kernel_v, [0,1])
        
        return output
    
# -----------------------------------------------------------------------------

class padRep(Layer):
    
    def __init__(self, im, depth, **kwargs):
        super(padRep, self).__init__(**kwargs)
        
        self.im = im # image / channel from input, that we want to extract and pad
        self.depth = depth # depth of padding on every side of input
        #         self.sess = tf.Session()

    def build(self, input_shape):
        super(padRep, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        i = 1
        d = self.depth
        res = x[:,:,:,self.im:self.im + 1]
        
        while d > 0 :
            if i <= d :
                res = tf.pad(res, [[0,0],[i,i],[i,i],[0,0]], "SYMMETRIC")
                d -= i
            else:
                res = tf.pad(res, [[0,0],[d,d],[d,d],[0,0]], "SYMMETRIC")
                d -= d
            i *= 2
        
        return res

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1] + 2 * self.depth, input_shape[2] + 2 * self.depth, 1)

In [34]:
filter_size = 51

input_img = keras.layers.Input((96,96,2))

x32 = keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(input_img)
x32 = keras.layers.AveragePooling2D((2,2),padding='same')(x32)

x64 = keras.layers.Conv2D(64,(3,3),activation='relu',padding='same')(x32)
x = keras.layers.UpSampling2D((2, 2), interpolation='bilinear')(x64)
x = keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(x)

k1h = keras.layers.Conv2D(filter_size,(3,3),activation='relu',padding='same')(x)
k1v = keras.layers.Conv2D(filter_size,(3,3),activation='relu',padding='same')(x)
k2h = keras.layers.Conv2D(filter_size,(3,3),activation='relu',padding='same')(x)
k2v = keras.layers.Conv2D(filter_size,(3,3),activation='relu',padding='same')(x)

im1 = padRep(0, filter_size//2)(input_img)
im2 = padRep(1, filter_size//2)(input_img)

out1 = SepConv()([im1,k1h,k1v])
out2 = SepConv()([im2,k2h,k2v])

output = keras.layers.Add()([out1,out2])

model = keras.Model(input_img, output)
model.compile(optimizer='adadelta', loss='mean_absolute_error')

model.summary()

[TensorShape([Dimension(None), Dimension(146), Dimension(146), Dimension(1)]), TensorShape([Dimension(None), Dimension(96), Dimension(96), Dimension(51)]), TensorShape([Dimension(None), Dimension(96), Dimension(96), Dimension(51)])]
here -  146
[TensorShape([Dimension(None), Dimension(146), Dimension(146), Dimension(1)]), TensorShape([Dimension(None), Dimension(96), Dimension(96), Dimension(51)]), TensorShape([Dimension(None), Dimension(96), Dimension(96), Dimension(51)])]
here -  146
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 96, 96, 2)    0                                            
__________________________________________________________________________________________________
conv2d_105 (Conv2D)             (None, 96, 96, 32)   608         input_16[0][0]                   
_______________

### Le test

In [35]:
L = SepConv()
sess = tf.Session()
L.build([[None,6,6,1],[None,4,4,3],[None,4,4,3]])

A = tf.reshape(tf.constant([[0,0,1,2,3,3],[0,0,1,2,3,3],[1,1,2,3,4,4],[2,2,3.,4,5,5],[3,3,4,5,6,6],[3,3,4,5,6,6]]),[1,6,6,1])
B = tf.reshape(tf.constant([[[0.,1,2],[0.,1,2],[0.,1,2],[0.,1,2]],[[0.,1,2],[0.,1,2],[0.,1,2],[0.,1,2]],[[0.,1,2],[0.,1,2],[0.,1,2],[0.,1,2]],[[0.,1,2],[0.,1,2],[0.,1,2],[0.,1,2]]]),[1,4,4,3])
C = tf.reshape(tf.constant([[[-1.,0,1],[-1.,0,1],[-1.,0,1],[-1.,0,1]],[[-1.,0,1],[-1.,0,1],[-1.,0,1],[-1.,0,1]],[[-1.,0,1],[-1.,0,1],[-1.,0,1],[-1.,0,1]],[[-1.,0,1],[-1.,0,1],[-1.,0,1],[-1.,0,1]]]),[1,4,4,3])


D = L.call([A,B,C])

[[None, 6, 6, 1], [None, 4, 4, 3], [None, 4, 4, 3]]
here -  6


In [36]:
print(sess.run(tf.shape(D)))
print(sess.run(D))

[1 4 4 1]
[[[[3.]
   [3.]
   [3.]
   [3.]]

  [[6.]
   [6.]
   [6.]
   [6.]]

  [[6.]
   [6.]
   [6.]
   [6.]]

  [[3.]
   [3.]
   [3.]
   [3.]]]]


In [79]:
M = np.array([[0,1,2],[2,3,4],[4,5,6]]).reshape([1,3,3,1])
x = model.predict(M)
print(M.reshape([3,3]))
print(x.reshape([9,9]))

[[0 1 2]
 [2 3 4]
 [4 5 6]]
[[0. 0. 0. 0. 1. 2. 2. 2. 2.]
 [0. 0. 0. 0. 1. 2. 2. 2. 2.]
 [0. 0. 0. 0. 1. 2. 2. 2. 2.]
 [0. 0. 0. 0. 1. 2. 2. 2. 2.]
 [2. 2. 2. 2. 3. 4. 4. 4. 4.]
 [4. 4. 4. 4. 5. 6. 6. 6. 6.]
 [4. 4. 4. 4. 5. 6. 6. 6. 6.]
 [4. 4. 4. 4. 5. 6. 6. 6. 6.]
 [4. 4. 4. 4. 5. 6. 6. 6. 6.]]
